In [405]:
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import seaborn as sns
import numpy as np
# Define panel widgets
import pandas as pd


In [406]:
tips = sns.load_dataset("tips")
bill_slider=pn.widgets.IntSlider(name='total_bill', start=0,end=int(max(tips.total_bill)),step=1, value=10)
bill_slider

IntSlider(end=50, name='total_bill', value=10)

## Data preprocessing

In [407]:
#tips = tips.fillna(0)
tips['smoker'] = np.where(tips['smoker']=='Yes',1, 0)
#tips['sex'] = np.where(tips['sex']=='Female',1, 0)
tips['time'] = np.where(tips['time']=='Dinner',20, 12)
tips=tips.astype({'total_bill':'int','tip':'int'}) 
tips

,total_bill,tip,sex,smoker,day,time,size
0,16,1,Female,0,Sun,20,2
1,10,1,Male,0,Sun,20,3
2,21,3,Male,0,Sun,20,3
3,23,3,Male,0,Sun,20,2
4,24,3,Female,0,Sun,20,4
...,...,...,...,...,...,...,...
239,29,5,Male,0,Sat,20,3
240,27,2,Female,1,Sat,20,2
241,22,2,Male,1,Sat,20,2
242,17,1,Male,0,Sat,20,2


In [408]:
tips=tips.interactive() ## super important

## Amount of tip versus total bill

In [409]:
y_axis_tip=pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['tip'],
    button_type='success')

In [411]:
sex=['Female','Male']
tip_pipeline=(
    tips[(tips.total_bill<=bill_slider) & (tips.sex.isin(sex))]
    .groupby(['sex','total_bill'])[y_axis_tip].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='total_bill')
    .reset_index(drop=True)
)

In [413]:
tip_plot=tip_pipeline.hvplot(x='total_bill',by='sex',y=y_axis_tip, title='bill vs tips')

In [414]:
tip_plot

## Tips table 

In [415]:
tip_table=tip_pipeline.pipe(pn.widgets.Tabulator, pagination='remote',page_size=10,sizing_mide='stretch_widths')

In [416]:
tip_table

## Scatter plot

In [417]:
tips_vs_bill_scatterplot_pipeline=(
     tips[(tips.total_bill<=bill_slider)]
    .groupby(['sex','total_bill','smoker'])[y_axis_tip].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='total_bill')
    .reset_index(drop=True)
)

In [418]:
tip_scatterplot=tips_vs_bill_scatterplot_pipeline.hvplot(x='total_bill',by='smoker',y=y_axis_tip, title='', size=80,kind='scatter',legend=True,height=500, width=500)

In [419]:
tip_scatterplot

##  Bar plot

In [439]:
yaxis_tip_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['smoker','time','total_bill'], 
    button_type='success'
)

tip_source_bar_pipeline = (
    tips[
        (tips.total_bill == bill_slider) &
        (tips.sex.isin(sex))
    ]
    .groupby(['total_bill','smoker','time'])[y_axis_tip].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='tip')  
    .reset_index(drop=True)
)

In [440]:
yaxis_tip_source

RadioButtonGroup(button_type='success', name='Y axis', options=['smoker', 'time', ...], value='smoker')

In [441]:
tip_source_bar_pipeline

In [442]:
tip_source_bar_plot = tip_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='tip', 
                                                     y=yaxis_tip_source,
                                                     title='Tip source')
tip_source_bar_plot

## Creating the dashboard 


In [449]:
#Layout using Template
template = pn.template.FastListTemplate(
    title="Waiter's tips dashboard", 
    sidebar=[pn.pane.Markdown("# Tips in restaurants"), 
             pn.pane.Markdown("#### Food servers tips in restaurants may be influenced by many factors, including the nature of the restaurant, size of the party, and table locations in the restaurant. Restaurant managers need to know which factors matter when they assign tables to food servers. For the sake of staff morale, they usually want to avoid either the substance or the appearance of unfair treatment of the servers, for whom tips (at least in restaurants in the United States) are a major component of pay."), 
             pn.pane.PNG('tips.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             bill_slider],
    main=[pn.Row(pn.Column(y_axis_tip, 
                           tip_plot.panel(width=700), margin=(0,25)), 
                 tip_table.panel(width=500)), 
          pn.Row(pn.Column(tip_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_tip_source, tip_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
#template.show()
template.servable();

Launching server at http://localhost:62138


In [446]:
# run on terminal
#panel serve Interactive_dashboard_with_panel.ipynb